# Confirm that noun banks are accessible by ipykernal.
#### - Shared Pandas dfs are transported as pickle files.
#### - Read using the %store -r '...' 'magic method'

In [81]:
# get noun banks
%store -r Animal
%store -r USA_State
%store -r all_countries
%store -r all_elements
%store -r greek_gods
%store -r roman_gods
%store -r cosmetic_items
%store -r office_supplies



no stored variable or alias USA_State



# Creating an engine will store a file "NounBankSQlite.db" at SQlite-Databases/NounBankSQlite.db
#### 

In [76]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine

DATABASE_LOCATION = "sqlite:///NounBankSQlite.db"

def post_to_sqlite():
    # establish database connection
    engine = create_engine(DATABASE_LOCATION)
    # populate database with noun banks
    Animal.to_sql('Animal', engine, if_exists='replace')
    USA_State.to_sql('USA_State', engine, if_exists='replace')
    all_countries.to_sql('all_countries', engine, if_exists='replace')
    all_elements.to_sql('all_elements', engine, if_exists='replace')
    greek_gods.to_sql('greek_gods', engine, if_exists='replace')
    roman_gods.to_sql('roman_gods', engine, if_exists='replace')
    cosmetic_items.to_sql('cosmetic_items', engine, if_exists='replace')
    office_supplies.to_sql('office_supplies', engine, if_exists='replace')

post_to_sqlite()



NameError: name 'US_State' is not defined

## Confirm that future updates to the Noun Bank do not raise any assertion errors

In [ ]:
import unittest as ut

class TestPostToSqlite(ut.TestCase):
    def __init__(self, *args, **kwargs):
        self.engine = create_engine(DATABASE_LOCATION) # create an engine to connect to the database for testing
        super(TestPostToSqlite, self).__init__(*args, **kwargs) # call the parent class's constructor

    def test_connecting_to_sqlite(self):
        self.assertEqual(self.engine.name, 'sqlite') # check that the engine is connected to a sqlite database

    def test_states_start_with_new(self):
        df = pd.read_sql('SELECT state FROM USA_State WHERE state LIKE "New%"', self.engine) # select all states that start with "New"
        self.assertEqual(df.values.tolist(), [['New Hampshire'], ['New Jersey'], ['New Mexico'], ['New York']]) # check that the correct states are returned
        
    def test_longest_len_greek_god(self):
        df = pd.read_sql('SELECT MAX(LENGTH(greek_god)), greek_god FROM greek_gods', self.engine)
        self.assertEqual(df['greek_god'][0], 'hephaestus' ) # check that the greek god with the longest name is Hephaestus
        self.assertEqual(df['MAX(LENGTH(greek_god))'][0], 10) # check that the length of the longest name is 11

    def test_element_with_largest_atomic_number(self):
        df = pd.read_sql('SELECT Element, AtomicNumber FROM all_elements WHERE AtomicNumber = (SELECT MAX(AtomicNumber) FROM all_elements)', self.engine)
        self.assertEqual(df['Element'][0], 'Oganesson') # check that the element with the largest atomic number is Oganesson
        self.assertEqual(df['AtomicNumber'][0], 118) # check that the atomic number of Oganesson is 118


#run the tests

if __name__ == '__main__':
    ut.main(argv=[''], verbosity=3, exit=False)

test_connecting_to_sqlite (__main__.TestPostToSqlite) ... ok
test_element_with_largest_atomic_number (__main__.TestPostToSqlite) ... ok
test_longest_len_greek_god (__main__.TestPostToSqlite) ... ok
test_states_start_with_new (__main__.TestPostToSqlite) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.017s

OK


In [ ]:
engine = create_engine(DATABASE_LOCATION)

import numpy as np


#use sql to view all available tables
available_tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', engine)

#choose a random table, never choosing the same table twice in a row
def choose_table():
    global current_table
    current_table = available_tables.sample(1).values[0][0]
    return current_table

FUCK = choose_table()

class Question:
    def __init__(self) -> None:
        self.antecedent = "Name a "
        self.NOUN = FUCK
        self.options ={ 
            'First & Last Letter Mode' : [" That Starts With The Letter", " That Ends With The Letter"],
            'Any Noun Word' : []
        }
    def get_question(self):
        return self.antecedent + self.NOUN + self.options['First & Last Letter Mode'][0] + " " + self.options['First & Last Letter Mode'][1]
q = Question()
print(q.get_question())



Name a usa_states That Starts With The Letter  That Ends With The Letter
